In [29]:
import seaborn as sns
import numpy as np
import pandas as pd
import mglearn as mg
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.preprocessing import OneHotEncoder , StandardScaler , LabelEncoder , OrdinalEncoder , TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier , KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import seaborn as sb
import category_encoders as ce

0- reading the data

In [30]:
aisles = pd.read_csv(r"archive\aisles.csv")
department = pd.read_csv(r"archive\departments.csv")
pro_prior = pd.read_csv(r"archive\order_products__prior.csv")
pro_train = pd.read_csv(r"archive/order_products__train.csv")
orders = pd.read_csv(r"archive\orders.csv")
products = pd.read_csv(r"archive\products.csv")

1- Joins and memory optimization

In [31]:
def reduce_memory(df):

    for col in df.columns:
        col_type = df[col].dtype

        if "int" in str(col_type):
            df[col] = df[col].astype("int32")

        elif "float" in str(col_type):
            df[col] = df[col].astype("float32")

        #بالبدايه حطيت بس else بعدين اكتشفت انه في انواع بيانات ثانيه مثل bool
        elif col_type == "object":
            df[col] = df[col].astype("category")

    return df

In [32]:
#حذفتهم لانهم ماخذين حجم كبير وبجميع الاحوال ما رح استخدمهم
del aisles["aisle"]
del department["department"]

dfs = [aisles , department , pro_prior , pro_train , orders , products]

for i in range(len(dfs)):
    dfs[i] = reduce_memory(dfs[i])

In [33]:
#   في m1 
#   pro_prior بلشنا ب هذول لانه يحتوي تفاصيل المنتجات داخل الطلبات
#   orders هو اللي بحتوي على معلومات الطلب , المستخدم , الوقت
m1 = pd.merge(pro_prior , orders , on = "order_id" , how = "left")
m2 = pd.merge(m1 , products , on = "product_id" , how = "left")
m3 = pd.merge(m2 , department , on = "department_id" , how = "left")
Full_DataSet = pd.merge(m3 , aisles , on = "aisle_id" , how = "left")

In [34]:
print(Full_DataSet.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 13 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int32   
 1   product_id              int32   
 2   add_to_cart_order       int32   
 3   reordered               int32   
 4   user_id                 int32   
 5   eval_set                category
 6   order_number            int32   
 7   order_dow               int32   
 8   order_hour_of_day       int32   
 9   days_since_prior_order  float32 
 10  product_name            category
 11  aisle_id                int32   
 12  department_id           int32   
dtypes: category(2), float32(1), int32(10)
memory usage: 1.5 GB
None


In [8]:
#كنت بدي اصير اعمل قراءه للملف مره ثانيه عشان ما اضل اعمل دمج كل ما اشغل الكود , بس اكتشفت انه تحميله بقعد وقت اكثر
#Full_DataSet.to_csv('archive/full_instacart_data.csv', index=False)

2- cleaning and preprocessing


In [35]:
print(Full_DataSet.isnull().sum())
print("---------------------------------------")

#كان هدفي اشوف ال نان من هون بس ما زبطت ف شفتها من الملف نفسه
Full_DataSet.head(10)
#اللي بين معي انه ال نان بكون موجود لكل اول اوردير بطلبه المستخدم\الزبون

order_id                        0
product_id                      0
add_to_cart_order               0
reordered                       0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_name                    0
aisle_id                        0
department_id                   0
dtype: int64
---------------------------------------


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13
5,2,17794,6,1,202279,prior,3,5,9,8.0,Carrots,83,4
6,2,40141,7,1,202279,prior,3,5,9,8.0,Original Unflavored Gelatine Mix,105,13
7,2,1819,8,1,202279,prior,3,5,9,8.0,All Natural No Stir Creamy Almond Butter,88,13
8,2,43668,9,0,202279,prior,3,5,9,8.0,Classic Blend Cole Slaw,123,4
9,3,33754,1,1,205970,prior,16,5,17,12.0,Total 2% with Strawberry Lowfat Greek Strained...,120,16


In [36]:
def check_outliers(FD , col):
    print(f"{col}: {FD[col].min()} , {FD[col].max()}")


In [37]:
#DF = pd.DataFrame(Full_DataSet)

colname = ["order_hour_of_day" , "days_since_prior_order" , "add_to_cart_order"]
for col in colname:
    check_outliers(Full_DataSet , col)
    print("\n")

Full_DataSet["add_to_cart_order"].describe()

order_hour_of_day: 0 , 23


days_since_prior_order: 0.0 , 30.0


add_to_cart_order: 1 , 145




count    3.243449e+07
mean     8.351076e+00
std      7.126671e+00
min      1.000000e+00
25%      3.000000e+00
50%      6.000000e+00
75%      1.100000e+01
max      1.450000e+02
Name: add_to_cart_order, dtype: float64

In [12]:
'''sns.boxplot(x=DF["days_since_prior_order"])
plt.show()'''


'sns.boxplot(x=DF["days_since_prior_order"])\nplt.show()'

In [13]:
'''sns.histplot(DF["order_hour_of_day"], bins=24, kde=True)
plt.show()'''


'sns.histplot(DF["order_hour_of_day"], bins=24, kde=True)\nplt.show()'

In [14]:
'''sns.histplot(DF["add_to_cart_order"], bins=50, kde=True)
plt.show()'''

#الرسوم البيانية (boxplot/histogram) لا تظهر Outliers غير منطقية، وتوزيع القيم متوافق مع طبيعة البيانات.
#لذلك لم يتم حذف أي صفوف.


'sns.histplot(DF["add_to_cart_order"], bins=50, kde=True)\nplt.show()'

In [38]:
Full_DataSet.dtypes

order_id                     int32
product_id                   int32
add_to_cart_order            int32
reordered                    int32
user_id                      int32
eval_set                  category
order_number                 int32
order_dow                    int32
order_hour_of_day            int32
days_since_prior_order     float32
product_name              category
aisle_id                     int32
department_id                int32
dtype: object

In [39]:
imputer = SimpleImputer(strategy = "constant" , fill_value = 0)

Full_DataSet["days_since_prior_order"] = imputer.fit_transform(Full_DataSet["days_since_prior_order"].values.reshape(-1,1))

DF = pd.DataFrame(Full_DataSet , columns = Full_DataSet.columns)

#DF.to_csv('archive/full_instacart_data.csv', index=False)
DF.head()



,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


In [40]:
product_counts = DF['product_name'].value_counts()
product_counts.head(20)

product_name
Banana                      472565
Bag of Organic Bananas      379450
Organic Strawberries        264683
Organic Baby Spinach        241921
Organic Hass Avocado        213584
Organic Avocado             176815
Large Lemon                 152657
Strawberries                142951
Limes                       140627
Organic Whole Milk          137905
Organic Raspberries         137057
Organic Yellow Onion        113426
Organic Garlic              109778
Organic Zucchini            104823
Organic Blueberries         100060
Cucumber Kirby               97315
Organic Fuji Apple           89632
Organic Lemon                87746
Apple Honeycrisp Organic     85020
Organic Grape Tomatoes       84255
Name: count, dtype: int64

In [41]:

numeric_cols = ["order_hour_of_day" , "days_since_prior_order" , "add_to_cart_order" , "order_number"]
low_cols = ["department_id" , "aisle_id" , "order_dow" , "eval_set"]
high_cols = ["user_id" , "product_id"]
target_cols = ["product_name"]

ONE = OneHotEncoder(handle_unknown='ignore' , sparse_output = False)
TE = ce.TargetEncoder(cols=high_cols , min_samples_leaf=100 , smoothing=10)
DF['product_freq'] = DF['product_name'].map(product_counts)

#تعلم كيف تعمل k fold cross validation مع target encoding
#ارجع تأكد من جزئية الاوتلايرز حاسها غلط او فيها نقص

